In [2]:
#OpenMM imports
from openmm import *
from openmm.app import *
from openmm.unit import *

#NumPy
import numpy as np

#Simulation output
from sys import stdout

# GROMACS to OpenMM Conversion

Here we attempt to build a molecular simulation with OpenMM using GROMACS coordinate and topology files. These files can easily be loaded with the GromacsGroFile() and GromacsTopFile() functions. The rest of the simulation is build as would normally be done with OpenMM. The parameter settings, such as temperature and cutoff, that would normally be given by a .mdp file in GROMACS are now specified in the script lines below. 

The top file often refers to other files that should be used in simulation, such as position restraints or forcefields. The IncludeDir= option specifies the path on which to look for these files. 

In [6]:
#Load Gro and Top files
gro = GromacsGroFile('Data/Input/System/glySPCE300K.gro')
top = GromacsTopFile('Data/Input/System/glycine.top', periodicBoxVectors=gro.getPeriodicBoxVectors(), includeDir='/usr/local/gromacs/share/gromacs/top')

#Create a system out of the topology by specifying bonds and interactions
system = top.createSystem(nonbondedMethod=PME, nonbondedCutoff=1*nanometer, constraints=HBonds)

#Specify integrator to use for advancing the system
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.004*picoseconds)

#Combine topology, system adn integrator to initialize the system
simulation = Simulation(top.topology, system, integrator)
simulation.context.setPositions(gro.positions)

#Do local energy minimization for stability
simulation.minimizeEnergy()

#Define output measures
simulation.reporters.append(PDBReporter('Gro_OMM_Conversion.pdb', 1000))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True, potentialEnergy=True, temperature=True))


In [5]:
#Run the simulation for a number of steps
simulation.step(10000)

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
1000,-76678.90208732928,289.76429251299294
2000,-75034.57407670742,294.83888217748495
3000,-74849.41201710749,295.9826900839883
4000,-75037.88464133014,298.222227372416
5000,-75078.38719000929,301.4259834578858
6000,-74901.40688792804,296.73965105983996
7000,-75484.3311681605,301.8301967578973
8000,-75273.53733990058,301.28522615126633
9000,-75319.9441177422,302.80378615530634
10000,-75425.04129296589,301.76095056693663


In [ ]:
import numpy as np
import simtk.openmm as mm
from simtk.openmm import app

# Load the system and integrator
pdb = app.PDBFile('system.pdb')
forcefield = app.ForceField('system.xml')
system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.PME,
                                 nonbondedCutoff=1.0*unit.nanometers, constraints=app.HBonds)
integrator = mm.LangevinIntegrator(300*unit.kelvin, 1.0/unit.picoseconds, 2.0*unit.femtoseconds)

# Define the pressure reporter
pressure_file = open('pressure.txt', 'w')
pressure_file.write('# Time (ps)\tPressure (bar)\n')
pressure_reporter = app.StateDataReporter(pressure_file, 1000, step=True, time=True,
                                           totalPressure=True, separator='\t')

# Create the simulation context
platform = mm.Platform.getPlatformByName('CPU')
simulation = app.Simulation(pdb.topology, system, integrator, platform)
simulation.context.setPositions(pdb.positions)

# Add the pressure reporter to the simulation
simulation.reporters.append(pressure_reporter)

# Run the simulation
simulation.step(10000)  # Run for 10,000 steps
